Load Packages

In [6]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time

# set path 

In [7]:
path = "C:/Users/liqia/Spring2021-Project3-group-3-master/data//train_set"
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

# read files

In [8]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [9]:
read_time_start=time.time()
X, Y = read_data(path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))

Read the original dataset takes 299.27 seconds


In [10]:
X.shape, Y.shape 

((3000, 3003), (3000,))

Data Preprocessing Imbalanced Dataset (SMOTE)-oversamplig

In [11]:
print('majority class: %d' % np.sum(Y == 0))
print('minority class: %d' % np.sum(Y == 1))
#imbalanced dataset

majority class: 2402
minority class: 598


In [12]:
def data_preprocessing(X, Y, path):

  distances = X
  y = Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [13]:
Balanced_X, Blanced_Y = data_preprocessing(X, Y, path)

In [14]:
Balanced_X.shape, Blanced_Y.shape

((4804, 3003), (4804,))

# split data

In [15]:

import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd
import random
random.seed(2021)

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(Balanced_X, Blanced_Y,test_size=0.2,random_state=0)

In [17]:
#split train and test with size=0.20.2

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size=0.2,random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2400, 3003), (2400,), (600, 3003), (600,))

# with images

In [4]:
import matplotlib.pyplot as plt
import scipy.io as scio
from PIL import Image
import pandas as pd
import numpy as np
#import cv2

from sklearn.metrics import classification_report, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
#from google.colab.patches import cv2_imshow
from imblearn.over_sampling import SMOTE
import pickle # to save the model

# models
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier